In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import pickle

In [ ]:
def clean_df(df_link):
    df = pd.read_parquet(df_link)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    df.drop(df.loc[df.duration < 0].index, inplace=True)
    df = df[(df.duration > 1) & (df.duration <= 60)].copy()
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [ ]:
df_train = clean_df('../data/yellow_tripdata_2023-01.parquet')
df_val = clean_df('../data/yellow_tripdata_2023-02.parquet')

In [ ]:
display(df_train.shape[0])
display(df_val.shape[0])

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [ ]:
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
lasso = Lasso(0.01)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
rid = Ridge()
rid.fit(X_train, y_train)

y_pred = rid.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
model = ElasticNet(alpha=0.1, l1_ratio=1)  # Parámetros alpha y l1_ratio
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open('../models/ridge_lreg.bin', 'wb') as f_out:
    pickle.dump((dv, rid), f_out)